# Defining Physics Lists

We need to provide a type to the `G4JLApplication` that represents the selected physics list. The easiest is to use one of the pre-packaged physics list (e.g. `FTFP_BERT`, `QGS_BIC`, `QBBC` and others can be added on demand). The toolkit will use the provided type to instantiate the physics list at the right moment during the initialization.

In [ ]:
using Geant4
using Test

## Using Standard Physics Lists
Lets play a bit one of these pre-packaged physics list

In [ ]:
fl = FTFP_BERT()
@test fl isa G4VUserPhysicsList
DumpList(fl)

It is normal that created instance by hand is empty. It needs to be created by the `G4RunManager` during initialization. Let's try to do it.

In [ ]:
app = G4JLApplication(
    physics_type = FTFP_BERT
)

In [ ]:
configure(app)
initialize(app)

In [ ]:
DumpList(app.physics)

## Defining a custom physics list
The way we can achieve it is by defining a dummy `struct` inheriting from `G4VUserPhysicsList` that provides a default constructor. In the constructor we can build the physics list and return an object that inherits from `G4VUserPhysicsList` to comply with the interface.

Here is an example:

In [ ]:
ui`/control/verbose 1`
ui`/run/verbose 1`

In [ ]:
struct MyPhysicsList <: G4VUserPhysicsList
    function MyPhysicsList(verbose)
        pl = G4VModularPhysicsList()
        RegisterPhysics(pl, move!(G4EmStandardPhysics(verbose)))  # make sure that the ownership is correctly transferred
        RegisterPhysics(pl, move!(G4OpticalPhysics(verbose)))     # by calling move!()
        return pl
    end 
end

In [ ]:
app = G4JLApplication(
    physics_type = MyPhysicsList,
    verbose=1,
)
configure(app)
initialize(app)

In [ ]:
DumpList(app.physics)